### Get Spotify credentials

In [13]:
# Get required info for token request
import os
from dotenv import load_dotenv
load_dotenv()
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
token_url = "https://accounts.spotify.com/api/token"

# Perform OAuth client credentials authorization
import requests
r = requests.post(
    "https://accounts.spotify.com/api/token",
    data={ "grant_type": "client_credentials" },
    auth=(client_id, client_secret)
)
r.raise_for_status()
response = r.json()
token = response["access_token"]
headers = { "Authorization": f"Bearer {token}" }

### Get Song a Day playlist data

In [14]:
playlists = { 2019: "6xJjyO5AGrZkyGkWYXfUX8", 2020: "3iNn4yKx9PxpSVYfRXkK8i" }
years = sorted(playlists.keys())
tracks = { year: [] for year in years }

# For each year's playlist, grab tracks
for year in years:
    # Page through track list for the playlist
    next_url = f"https://api.spotify.com/v1/playlists/{playlists[year]}/tracks"
    while next_url:
        # Fetch the current page
        r = requests.get(next_url, headers=headers)
        r.raise_for_status()
        response = r.json()
        # Reshape each playlist item and add to our "tracks" list for the relevant year
        for playlist_item in response["items"]:
            tracks[year].append({
                "name": playlist_item["track"]["name"],
                "id": playlist_item["track"]["id"],
                "album_id": playlist_item["track"]["album"]["id"],
                "album_name": playlist_item["track"]["album"]["name"],
                "album_images": dict([
                    ({640: "large", 300: "medium", 64: "small"}[i["width"]], i["url"])
                    for i in playlist_item["track"]["album"]["images"]
                ]),
                "artists": [
                    { "name": artist["name"], "id": artist["id"] }
                    for artist in playlist_item["track"]["artists"]
                ],
                "primary_artist_name": playlist_item["track"]["artists"][0]["name"],
                "primary_artist_id": playlist_item["track"]["artists"][0]["id"],
                "is_local": playlist_item["is_local"]
            })
        # Take note of the URL of the next page of playlist items
        next_url = response["next"]

In [15]:
# Compile all tracks
all_tracks = []
# Assign a date to each track
from datetime import datetime, timedelta
for year in years:
    curr_date = datetime(year, 1, 1)
    for track in tracks[year]:
        track["date"] = curr_date
        all_tracks.append(track) # add to big array
        curr_date += timedelta(days=1)

print(tracks[2019][-1])

{'name': 'Northstar (feat. Vic Mensa & Maceo Haymes)', 'id': '2rQo6riaUQ9KRzDgPLqeWx', 'album_id': '1H2Y4lomFedlGP6yEd15cT', 'album_name': 'Opia', 'album_images': {'large': 'https://i.scdn.co/image/ab67616d0000b2732fe16c52fdb5c238f66586c1', 'medium': 'https://i.scdn.co/image/ab67616d00001e022fe16c52fdb5c238f66586c1', 'small': 'https://i.scdn.co/image/ab67616d000048512fe16c52fdb5c238f66586c1'}, 'artists': [{'name': 'Malcolm London', 'id': '1ikfT4dtC0eCVk4TZdfHGO'}], 'primary_artist_name': 'Malcolm London', 'primary_artist_id': '1ikfT4dtC0eCVk4TZdfHGO', 'is_local': False, 'date': datetime.datetime(2019, 12, 31, 0, 0)}


In [16]:
import pandas as pd

df = pd.DataFrame(all_tracks)
df

,name,id,album_id,album_name,album_images,artists,primary_artist_name,primary_artist_id,is_local,date
0,So It Goes,0EA2RhRHL4KWeNa7JfD1Yw,5wtE5aLX5r7jOosmPhJhhk,Swimming,{'large': 'https://i.scdn.co/image/ab67616d000...,"[{'name': 'Mac Miller', 'id': '4LLpKhyESsyAXpc...",Mac Miller,4LLpKhyESsyAXpc4laK94U,False,2019-01-01
1,Fertilizer,6jypaMkKsoc5npsVzxhksl,392p3shh2jkxUxY2VHvlH8,channel ORANGE,{'large': 'https://i.scdn.co/image/ab67616d000...,"[{'name': 'Frank Ocean', 'id': '2h93pZq0e7k5yf...",Frank Ocean,2h93pZq0e7k5yf4dywlkpM,False,2019-01-02
2,Brain Damage,05uGBKRCuePsf43Hfm0JwX,4LH4d3cOWNNsVw41Gqt2kv,The Dark Side of the Moon,{'large': 'https://i.scdn.co/image/ab67616d000...,"[{'name': 'Pink Floyd', 'id': '0k17h0D3J5Vfsdm...",Pink Floyd,0k17h0D3J5VfsdmQ1iZtE9,False,2019-01-03
3,Jet Fuel,2JmFRXaJrkAUD2cs6U20KG,5wtE5aLX5r7jOosmPhJhhk,Swimming,{'large': 'https://i.scdn.co/image/ab67616d000...,"[{'name': 'Mac Miller', 'id': '4LLpKhyESsyAXpc...",Mac Miller,4LLpKhyESsyAXpc4laK94U,False,2019-01-04
4,Come Back to Earth,01z2fBGB8Hl3Jd3zXe4IXR,5wtE5aLX5r7jOosmPhJhhk,Swimming,{'large': 'https://i.scdn.co/image/ab67616d000...,"[{'name': 'Mac Miller', 'id': '4LLpKhyESsyAXpc...",Mac Miller,4LLpKhyESsyAXpc4laK94U,False,2019-01-05
...,...,...,...,...,...,...,...,...,...,...
648,Ivy,2ZWlPOoWh0626oTaHrnl2a,3mH6qwIy9crq0I9YQbOuDf,Blonde,{'large': 'https://i.scdn.co/image/ab67616d000...,"[{'name': 'Frank Ocean', 'id': '2h93pZq0e7k5yf...",Frank Ocean,2h93pZq0e7k5yf4dywlkpM,False,2020-10-10
649,Everybody,1OuN92HcVG6NVpWbeESNB3,5sY6UIQ32GqwMLAfSNEaXb,Circles,{'large': 'https://i.scdn.co/image/ab67616d000...,"[{'name': 'Mac Miller', 'id': '4LLpKhyESsyAXpc...",Mac Miller,4LLpKhyESsyAXpc4laK94U,False,2020-10-11
650,100 Grandkids,4z0PnuB07fxtVZZRWsCfxb,2Tyx5dLhHYkx6zeAdVaTzN,GO:OD AM,{'large': 'https://i.scdn.co/image/ab67616d000...,"[{'name': 'Mac Miller', 'id': '4LLpKhyESsyAXpc...",Mac Miller,4LLpKhyESsyAXpc4laK94U,False,2020-10-12
651,Hold Me Down,1Mza2sr6tPhy6jjI3HB9fW,3xybjP7r2VsWzwvDQipdM0,Freudian,{'large': 'https://i.scdn.co/image/ab67616d000...,"[{'name': 'Daniel Caesar', 'id': '20wkVLutqVOY...",Daniel Caesar,20wkVLutqVOYrc0kxFs7rA,False,2020-10-13
